# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongodb://127.0.0.1:27017
`

In [90]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [91]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [92]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [93]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [94]:
# Review the collections in our new database
print(db.list_collection_names())

['establishments']


In [95]:
# review a document in the establishments collection
from pprint import pprint
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677cac2d18a16467218f5d35'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'latitude': 0.0,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'longit

In [96]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [97]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 Greenwich High Street",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE10 8NY",
    "PhoneNumber": "020 8123 4567",
    "RatingValue": None,
    "RatingDate": None,
    "LocalAuthorityName": "Greenwich",
    "BusinessTypeID": None
}

In [98]:
# Insert the new restaurant into the collection
db.establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('678466cf3a109ebe59846f21'), acknowledged=True)

In [99]:
# Check that the new restaurant was inserted
from pprint import pprint
pprint(db.establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': '123 Greenwich High Street',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'Greenwich',
 'PhoneNumber': '020 8123 4567',
 'PostCode': 'SE10 8NY',
 'RatingDate': None,
 'RatingValue': None,
 '_id': ObjectId('677cae4c5ff1bcb946563d46'),
 'latitude': 0.0,
 'longitude': 0.0}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [100]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [101]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and assign it to a variable
result = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "_id": 0}
)
business_type_id = result["BusinessTypeID"]

# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [102]:
# Confirm that the new restaurant was updated
from pprint import pprint
pprint(db.establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': '123 Greenwich High Street',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'Greenwich',
 'PhoneNumber': '020 8123 4567',
 'PostCode': 'SE10 8NY',
 'RatingDate': None,
 'RatingValue': None,
 '_id': ObjectId('677cae4c5ff1bcb946563d46'),
 'latitude': 0.0,
 'longitude': 0.0}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [103]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")

Number of documents with LocalAuthorityName as 'Dover': 0


In [104]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of documents deleted: {delete_result.deleted_count}")

Number of documents deleted: 0


In [105]:
# Check if any remaining documents include Dover
remaining_dover = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of remaining documents with LocalAuthorityName as 'Dover': {remaining_dover}")

Number of remaining documents with LocalAuthorityName as 'Dover': 0


In [106]:
# Check that other documents remain with 'find_one'
remaining_docs = db.establishments.find_one()
print("A sample remaining document in the collection:")
pprint(remaining_docs)

A sample remaining document in the collection:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677cac2d18a16467218f5d35'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'latitude': 0.0,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [114]:
# Change the data type from String to Decimal for longitude and latitude
# Convert latitude and longitude from strings to decimal numbers
establishments = db["establishments"]

# Fetch all documents and update latitude and longitude
for document in establishments.find({"$or": [{"latitude": {"$type": "string"}}, {"longitude": {"$type": "string"}}]}):
    # Parse strings into floats
    lat = float(document["latitude"])
    lon = float(document["longitude"])
    
    # Update the document with converted values
    establishments.update_one(
        {"_id": document["_id"]},
        {"$set": {"latitude": lat, "longitude": lon}}
    )
print("Converted latitude and longitude to decimal numbers.")


Converted latitude and longitude to decimal numbers.


Use `update_many` to convert `RatingValue` to integer numbers.

In [113]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [109]:
# Change the data type from String to Integer for RatingValue
# Fetch all documents with RatingValue as string
for document in establishments.find({"RatingValue": {"$type": "string"}}):
    # Convert RatingValue to integer
    rating = int(document["RatingValue"])
    
    # Update the document with the converted value
    establishments.update_one(
        {"_id": document["_id"]},
        {"$set": {"RatingValue": rating}}
    )
print("Converted RatingValue to integer numbers.")

Converted RatingValue to integer numbers.


In [110]:
# Check for documents with latitude or longitude as strings
string_lat_count = establishments.count_documents({"latitude": {"$type": "string"}})
string_lon_count = establishments.count_documents({"longitude": {"$type": "string"}})

print(f"Documents with latitude as string: {string_lat_count}")
print(f"Documents with longitude as string: {string_lon_count}")


Documents with latitude as string: 0
Documents with longitude as string: 0


In [111]:
establishments.update_many(
    {"latitude": {"$exists": False}},
    {"$set": {"latitude": 0.0}}
)

establishments.update_many(
    {"longitude": {"$exists": False}},
    {"$set": {"longitude": 0.0}}
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [112]:
# Check that the coordinates and rating value are now numbers
lat_check = establishments.find_one({"latitude": {"$type": "double"}})
lon_check = establishments.find_one({"longitude": {"$type": "double"}})

if lat_check and lon_check:
    print("All latitude and longitude values are now numbers.")
else:
    print("Some latitude or longitude values are still not numbers.")


All latitude and longitude values are now numbers.
